In [1]:
import numpy as np
import pandas as pd
import datetime
from sqlalchemy import create_engine
import plotly.express as px
from database_utils import add_new_unl, add_new_dot, clean_unleashed, clean_dot, get_level_2

In [2]:
# database connection

db_password = "UnitCircle42!"
db_user = "postgres"
db_name = "dot"
endpoint = "awakedb.cre3f7yk1unp.us-west-1.rds.amazonaws.com"

connection_string = f"postgresql://{db_user}:{db_password}@{endpoint}:5432/{db_name}"
engine = create_engine(connection_string)

In [20]:
## PASTE 2 FILES BELOW AND UPDATE ALL TABLES

unl_download = r"C:\Users\mikej\Downloads\SalesEnquiryList - 2024-03-21T065414.584.xlsx" ### file download from unleashed
dot_download = r"C:\Users\mikej\Downloads\SHOP Supplier Reporting - Invoice Details Excel - 2024-03-21T065444.531.xlsx"  ### file download from dot

# add_new_unl(unl_download)  ## adds new direct purchases to raw database
add_new_dot(dot_download)  ## adds new indirect purchases to raw database

# clean_unleashed()          ## clean raw direct purchases and add to clean directdb
clean_dot()                ## clean raw indirect purchases and add to clean indirect db
get_level_2()              ## combine clean direct/indirect and populate level_2 db  --  dot invisible

In [21]:
# create level_1.csv for internal use

level_1 = pd.read_sql("SELECT * FROM unleashed_raw WHERE completed_date > '2022-12-31';", con = engine)
level_1.to_csv(r"C:/Users/mikej/Desktop/cpg-sales/data/l1.csv", index=False)

In [22]:
### updating all_sales_data.csv in "data" directory

level_2 = pd.read_sql("SELECT * FROM level_2 WHERE date > '2020-12-31';", con = engine)
level_2 = level_2[level_2.market_segment != 'Samples']
level_2.to_csv(r"C:/Users/mikej/Desktop/cpg-sales/data/all_sales_data.csv", index=False)

In [23]:
level_2.head()

date sale_origin         market_segment             parent_customer  \
0  2024-03-20         dot       Alternate Retail  Kelli's Gift Shop Supplies   
1  2024-03-20         dot  Broadline Distributor                    US Foods   
2  2024-03-20         dot       Alternate Retail  Kelli's Gift Shop Supplies   
3  2024-03-20         dot       Alternate Retail  Kelli's Gift Shop Supplies   
4  2024-03-20         dot       Alternate Retail  Kelli's Gift Shop Supplies   

                    customer  \
0  KELLI'S GIFT SHOP SUPPLIE   
1  USF/CONNECTICUT-USF DIREC   
2  KELLI'S GIFT SHOP SUPPLIE   
3  KELLI'S GIFT SHOP SUPPLIE   
4  KELLI'S GIFT SHOP SUPPLIE   

                                                item   qty      usd      cad  \
0                CAFFEINATED CHOCOLATE BITES SINGLES -14.0 -1579.20 -2100.34   
1  AWAKE CHOCOLATE AWAKE CAFF MILK CHOC- 6X12PK M...   1.0    85.54   113.77   
2  AWAKE CHOCOLATE AWAKE CAFF DARK CHOC- 6X12PK M...  -4.0  -273.84  -364.21   
3        CAFFEINATED CHOCOLATE BITES SINGLES CARAMEL -18.0 -2030.40 -2700.43   
4           CAFFEINATED CHOCOLATE BITES SINGLES DARK -16.0 -1804.80 -2400.38   

   month  year  
0  March  2024  
1  March  2024  
2  March  2024  
3  March  2024  
4  March  2024

In [24]:
level_1.head()

order_num  order_date    req_date completed_date           warehouse  \
0  SO-00140851  2023-12-21  2023-12-22     2023-12-22      Complemar - PA   
1  SO-00140850  2023-12-21  2023-12-22     2023-12-22  Complemar - Nevada   
2  SO-00140849  2023-12-21  2023-12-22     2023-12-22      Complemar - PA   
3  SO-00140848  2023-12-21  2023-12-22     2023-12-22      Complemar - PA   
4  SO-00140847  2023-12-21  2023-12-22     2023-12-22  Complemar - Nevada   

  customer_name customer_type                          product  \
0     Amazon SC        Amazon  50ct Pouch - PB Milk Choc Bites   
1     Amazon SC        Amazon  50ct Pouch - PB Milk Choc Bites   
2     Amazon SC        Amazon  50ct Pouch - PB Milk Choc Bites   
3     Amazon SC        Amazon  50ct Pouch - PB Milk Choc Bites   
4     Amazon SC        Amazon  50ct Pouch - PB Milk Choc Bites   

    product_group     status  quantity  sub_total  
0  Finished Goods  Completed       1.0      46.65  
1  Finished Goods  Completed       1.0      46.65  
2  Finished Goods  Completed       1.0      46.65  
3  Finished Goods  Completed       1.0      46.65  
4  Finished Goods  Completed       1.0      46.65

In [25]:
### bar graph should populate if all is well

# level_2 = pd.read_sql('SELECT * FROM level_2;', con = engine)
level_2.date = pd.to_datetime(level_2.date)

px.bar(level_2.set_index('date').groupby(pd.Grouper(freq='M'))['usd'].sum(),
        y='usd',
        template='plotly_white',
        labels={'usd':'$USD',
                'date':''},
        color='usd',
        width=900,
        text_auto=",.2s",
        height=400).update(layout_coloraxis_showscale=False).update_traces(textposition='outside')

In [83]:
### bar graph should populate if all is well
# DAILY MTD

# level_2 = pd.read_sql('SELECT * FROM level_2;', con = engine)
level_2.date = pd.to_datetime(level_2.date)

px.bar(level_2[level_2.date>'2024-01-31'].set_index('date').groupby(pd.Grouper(freq='d'))['usd'].sum(),
        y='usd',
        template='plotly_white',
        labels={'usd':'$USD',
                'date':''},
        color='usd',
        width=900,
        color_continuous_scale='Oranges',
        text_auto=",.2s",
        height=400).update(layout_coloraxis_showscale=False).update_traces(textposition='outside')

In [27]:
level_1.completed_date.max()

datetime.date(2024, 3, 20)

In [28]:
level_2.date.max()

Timestamp('2024-03-20 00:00:00')

In [29]:
# MAKE LEVEL_2 INTO TABLEU PREP OUTPUT

In [30]:
# load in level_2 and check columns

level_2 = pd.read_sql("SELECT * FROM level_2 WHERE date > '2020-12-31';", con = engine)

tab = level_2.copy().drop(columns=['qty','cad','month','year'])
tab.columns

# fix columns to mimic tableau download data output

tab.columns = ['Invoice Date', 'Sale Origin', 'Market Segment', 'Parent Customer', 'Customer', 'Item Full Description', 'Dollars', ]

# add vistar retail y/n column

tab['Vistar Retail'] = np.where(tab['Parent Customer'] == 'Vistar Retail', str('Yes'), str('No'))

tab.to_csv(r"C:\Users\mikej\Desktop\cpg-sales\data\all_sales_data_tableau.csv", index=False)

In [31]:
# make unleashed raw tableau-ready (level_1)
# load in level_1 and check columns

level_1 = pd.read_sql("SELECT * FROM unleashed_clean WHERE completed_date > '2020-12-31';", con = engine)


# load in level_1 and check columns

tab = level_1.copy().drop(columns=['month','year'])

# fix columns to mimic tableau download data output

tab.columns = ['Customer Name', 'Invoice Date','Item Full Description', 'Quantity','Sub Total', 'Dollars', 'Table','Market Segment', 'Parent Customer']

tab['Table'] = 'Unleashed'

tab.to_csv(r"C:\Users\mikej\Desktop\cpg-sales\data\level_1_tableau.csv", index=False)

In [32]:
l1 = pd.read_sql("SELECT * FROM unleashed_raw WHERE completed_date > '2022-12-31';", con = engine)
px.scatter(l1[l1.customer_name=='DOT Foods, Inc.'][['completed_date','sub_total']].set_index('completed_date').sort_index())

In [33]:
px.bar(pd.DataFrame(level_1.groupby('parent_customer')['usd'].sum()))

In [34]:
### bar graph should populate if all is well

l1.completed_date = pd.to_datetime(l1.completed_date)
l1['usd'] = l1['sub_total']*.75

l1_bar_df = l1.groupby('completed_date')['usd'].sum().reset_index().set_index('completed_date')
l1_bar_df =round(l1_bar_df[l1_bar_df.index>'2024-01-31']).sort_index()

level_1_bar = px.bar(l1_bar_df,
                     y='usd',
                     template='plotly_white',
                     labels={'usd':'',
                             'completed_date':''},
                     height=325,
                     text_auto=",.2s")

level_1_bar.update_traces(hovertemplate = '$%{y:.2s}'+'<br>%{x:%Y-%m-%d}<br>')
level_1_bar.update_traces(marker_color='rgb(239, 83, 80)')
level_1_bar.update_coloraxes(showscale=False)
level_1_bar.update_yaxes(showticklabels=False,showgrid=True,tickprefix='$',gridcolor="#B1A999",tickvals=[0,25000,50000,75000,100000],tickfont=dict(color='#5A5856', size=14))
level_1_bar.update_xaxes(showgrid=False,gridcolor='gray',tickfont=dict(color='#5A5856', size=13),title_font=dict(color='#5A5856',size=15))
level_1_bar.update_xaxes(tickmode='array',tickvals = l1_bar_df.index, ticktext=l1_bar_df.index.strftime('<b>%a<br>%d</b>'))
level_1_bar.update_layout(hoverlabel=dict(font_size=18,font_family="Rockwell"))

In [35]:
l1.groupby(l1.completed_date).sub_total.sum().reset_index().set_index('completed_date')

sub_total
completed_date           
2023-01-03         182.61
2023-01-05         197.28
2023-01-10        1158.38
2023-01-11         785.14
2023-01-12        3009.91
...                   ...
2024-03-13       10749.41
2024-03-14       23825.43
2024-03-18         248.63
2024-03-19        3547.41
2024-03-20           0.00

[306 rows x 1 columns]

In [79]:
kellis = level_2[(level_2.parent_customer=="Kelli's Gift Shop Supplies") & (level_2.usd < 0)]
px.scatter(kellis.sort_values('date',ascending=False)[['date','usd']],
      x='date',
      y='usd')

In [88]:
level_2.date = pd.to_datetime(level_2.date)

px.bar(level_2[level_2.parent_customer=="Kelli's Gift Shop Supplies"].set_index('date').groupby(pd.Grouper(freq='W'))['usd'].sum())